In [1]:
import pandas as pd
import numpy as np
import glob
import xarray as xr

idx = pd.IndexSlice
_NASA_data_path = r'/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/Data/Nasa Data/portal.nccs.nasa.gov/datashare/gmao/geos-cf/v1/das/Y2021/M01/D08' # use your path
airquality_files = glob.glob(_NASA_data_path + "/GEOS-CF.v01.rpl.aqc_tavg*.nc4")
meteorological_files = glob.glob(_NASA_data_path + "/GEOS-CF.v01.rpl.met_tavg*.nc4")

_thudohanoi_data_path = r'/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/Data/thudohanoi'
_thudohanoi_files = glob.glob(_thudohanoi_data_path + '/*.csv')

In [2]:
# ds = xr.open_dataset('./das/Y2018/M01/D01/GEOS-CF.v01.rpl.aqc_tavg_1hr_g1440x721_v1.20180101_0030z.nc4')
# df = ds.to_dataframe()

# ds_2 = xr.open_dataset('./das/Y2018/M01/D01/GEOS-CF.v01.rpl.met_tavg_1hr_g1440x721_x1.20180101_1030z.nc4')
# df_2 = ds_2.to_dataframe()

# ds_3 = xr.open_dataset('./MERRA2.20170101.A1.05x0625.AS.nc4')
# df_3 = ds_3.to_dataframe()

In [10]:
thudohanoi_df = pd.DataFrame()
for file in _thudohanoi_files:
    thudohanoi_df = thudohanoi_df.append(pd.read_csv(file))

In [4]:
# def process_thudohanoi_data(thudohanoi_df):
#     thudohanoi_df.reset_index(drop=True,inplace=True)
#     thudohanoi_df = thudohanoi_df.groupby(['site_id', 'time', 'parameter']).sum()
#     return thudohanoi_df
# tmp_df = process_thudohanoi_data(thudohanoi_df)

KeyError: 'parameter'

In [2]:
def read_site_data():
    site_df = pd.read_csv(_thudohanoi_data_path + "/Site_data/VN_site_data.csv")
    site_df.set_index('id', inplace=True)
    return site_df

def filter_location_data(df):
    #Dau tien la filter cac location tren moitruongthudo
    #Tiep theo la lay du lieu tren nasa cua cac filter location
    site_df = read_site_data()
    df.sort_index(level=['lat', 'lon'], ascending=[1,1], inplace=True)
    df = df.loc[idx[21, 105.75,:]]
    return df

def filter_meteorological_data(df):
    df = df.reset_index(level=1, drop=True)
    df = df[['RH', 'T', 'TPREC', 'U', 'V']]
    df.rename(columns={"RH": "Relative Humidity", "T": "Temperature", "TPREC": "Precipitation",
                       "U": "Eastward wind", "V":"Northward wind"}, inplace=True)
    return df
def filter_airquality_data(df):
    df = df.reset_index(level=1, drop=True)
    df.rename(columns={'PM25_RH35_GCC': 'PM25'}, inplace=True)
    return df


full_airquality_df = pd.DataFrame()
for file in airquality_files:
    ds = xr.open_dataset(file)
    full_airquality_df = full_airquality_df.append(filter_airquality_data(ds.to_dataframe()))

airquality_df = filter_location_data(full_airquality_df)

full_meteorological_df = pd.DataFrame()
for file in meteorological_files:
    ds = xr.open_dataset(file)
    full_meteorological_df = full_meteorological_df.append(filter_meteorological_data(ds.to_dataframe()))

meteorological_df = filter_location_data(full_meteorological_df)

In [29]:
#converse all units to ug/m3
#all units but PM2.5 is mol/molair
#Conversion method is from this article
def converse_units(volume_mixing_ratio, polutant_name):
    #Determine molecular mass of polutant
    #Taken from Nasa document for data
    switcher = {
        'CO': 28.00,
        'NO2': 46.00,
        'O3': 48.00,
        'SO2': 64.00
    }
    molecular_weight = switcher.get(polutant_name, np.nan)
    
    #change mol/mol air to ppm (10^-6 mol/mol air = 1 ppm)
    ppm = volume_mixing_ratio / 10**-6
    #Change ppmv to mg/m^3
    result = 0.0409 * ppm * molecular_weight
    #from kg to ug
    result = result * 1000
    return result

for polutant_name in ['CO','NO2','O3','SO2']:
    airquality_df.loc[:, polutant_name] = airquality_df.loc[:, polutant_name].apply(lambda x: converse_units(x, polutant_name))


/home/nam/Development/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [ ]:
#https://vanbanphapluat.co/quyet-dinh-1459-qd-tcmt-2019-ky-thuat-tinh-toan-va-cong-bo-chi-so-chat-luong-khong-khi-viet-nam
def create_BP_df():
    BP_Ii = [0,50,100,150,200,300,400,500]
    BP_O3_1h = [0,160,200,300,400,800,1000,1200]
    BP_O3_8h = [0,120,170,210,400]
    BP_CO = [0,10000,30000,45000,60000,90000,120000,150000]
    BP_SO2 = [0,125,350,550,800,1600,2100,2630]
    BP_NO2 = [0,100,200,700,1200,2350,3100,3850]
    BP_PM10 = [0,50,150,250,350,420,500,600]
    BP_PM25 = [0,25,50,80,150,250,350,500]
    BP = pd.DataFrame(data = [range(1,9),BP_Ii, BP_O3_1h, BP_O3_8h, BP_CO, BP_SO2, BP_NO2, BP_PM10, BP_PM25],
                      index=['I','Ii','O3','O3_8h','CO','SO2','NO2','PM10','PM25'], dtype= np.int64)
    BP = BP.transpose().set_index('I')
    #Add a upper to calculate I = 8
    BP.loc[9] = [999999,999999,999999,999999,999999,999999,999999,999999]
    BP.index = BP.index.astype(np.int64)
    return BP
global_BP = create_BP_df()

In [ ]:
#Cho vao bang BP, series gia tri 5 hop chat
#Dau ra la 1 series gom co gia tri index cua 5 hop chat do
#Dau ra lan luot cua cac chat [CO, NO2, O3, PM25, SO2]
def calculate_BP_I(polutant_data, BP = create_BP_df()):
    result_I = []
    BP_polutant_column = BP.loc[:,polutant_data.name]
    result_I = polutant_data.apply(lambda x: BP_polutant_column.ge(x).idxmax() - 1) 
    return result_I
sample_series = airquality_df
sample_series = sample_series.apply(lambda x: calculate_BP_I(x,global_BP), axis=0, result_type = 'broadcast')
# sample_result = pd.DataFrame(sample_result, index=['lat', 'lon', 'time'])

In [ ]:
#Cong thuc tinh duoc lay tu duong link nay: 
#http://tapchimoitruong.vn/pages/article.aspx?item=New-calculation-of-the-Vi%E1%BB%87t-Nam-Air-Quality-Index-51306
#http://cem.gov.vn/storage/news_file_attach/QD%201459%20TCMT%20ngay%2012.11.2019%20AQI.pdf
#Cong thuc nay se lay input la 1 dataframe so do hop chat cho output ra 1 series gom so AQI cua moi gio (Lon nhat trong tat ca hop chat)
def calculate_AQI_h(data = None, formular = 1):  
    #Cong thuc 1 = ((Ii[i+1] - Ii[i])/ (BP[i+1] - BP[i]))* (C[h] - BP[i]) +Ii[i]
    def calculate_AQI_formular(polutant_data, I, BP = global_BP):
        result = []
        for data in zip(polutant_data.index, polutant_data.values, I.values):
            polutant_name, polutant_value, I = data
            I = int(I)
            try:
                AQI = ((BP.loc[I + 1,'Ii'] - BP.loc[I, 'Ii']) / (BP.loc[I+1, polutant_name] - BP.loc[I, polutant_name])
                                                   * (polutant_value - BP.loc[I, polutant_name])) + BP.loc[I, 'Ii']
            except KeyError:
                print(data)
                if I == 0:
                    AQI = 0
                else:
                    AQI = 500
            result.append(AQI)
        return result
    
    #VietNam system use 12 hours for nowcast, the current hour and 11 hours before.
    def nowcast(polutant_data, element):
        lat, lon, time = element.name
            #Nhung cong viec can lam
            #Get data for the hour and 12 hours behind it in the same spot
        tmp_12h_data_storage = polutant_data.loc[
            idx[lat, lon, time - pd.Timedelta(hours=12) : time]]
            #Kiem tra xem trong 3 gio gan nhat thi it nhat 2 gio phai co so lieu
            #Neu khong qua duoc check o tren thi so lieu tinh duoc dat la np.nan
        if tmp_12h_data_storage.iloc[:3].isna().sum() > 2:
            result = np.nan
        elif len(tmp_12h_data_storage) < 4:
            result = np.nan
        else:
            #Trong 12 gia tri do lay gia tri min va max
            min_data = tmp_12h_data_storage.min()
            max_data = tmp_12h_data_storage.max()
            #Tinh Weight bang cach lay gia tri lon nhat chia gia tri nho
            w = min_data / max_data
            #Neu weight < 1/2 thi dat weight = 1/2
            if w < 1/2:
                w = 1/2
            #Tinh gia tri nowcast dua theo cong thuc
            nowcast = sum([(w**i) * tmp_12h_data_storage.iloc[i-1] for i in range(1,len(tmp_12h_data_storage)+1)]) / sum([(w**i) for i in range(1,len(tmp_12h_data_storage) + 1)])
            #Add gia tri now cast vao ket qua tra ve
            result = nowcast
            print(tmp_12h_data_storage)
        return result
    #Init variable
    idx = pd.IndexSlice
    #Sort index to slice
    data.sort_index(level=['lat','lon', 'time'], ascending=[1,1,1], inplace=True)
    
    #Air index number of all polutant data
    
    data.loc[:,['PM25']] = data.loc[:,['PM25']].apply((lambda x: nowcast(data.loc[:,'PM25'], x)), axis=1, result_type='broadcast')
    I_number = data.apply((lambda x: calculate_BP_I(x, global_BP)), axis=0, result_type='broadcast')
    
    calculated_AQI = I_number.apply((lambda x: calculate_AQI_formular(data.loc[x.name, :], x, global_BP)),
                                axis=1, result_type='broadcast')
    calculated_AQI['AQI_h'] = calculated_AQI.max(axis=1)
    calculated_AQI['AQI_h_Polutant'] = calculated_AQI.idxmax(axis=1)
    calculated_AQI['AQI_h_I'] = I_number.max(axis=1)  
    return calculated_AQI
AQI = calculate_AQI_h(airquality_df)

In [ ]:
# '''Lam 4 model:
# Random forest
# gradient boosting regression
# decision tree regression
# mlp regression'''

import numpy as np
from sklearn import datasets
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

moon_dataset = make_moons(1000, noise = 0.4)
X = moon_dataset[0]
y = moon_dataset[1]

rnd_clf = RandomForestRegressor(n_estimators = 500, max_leaf_nodes = 16, n_jobs = -1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

rnd_clf.fit(X_train, y_train)


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators=3, learning_rate = 1.0)
gbrt.fit(X_train, y_train)

#measure the validation error at each stage of training to find the optimal number of trees

errors = [mean_squared_error(y_val, y_pred)
         for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)

gbrt_best = GradientBoostingRegressor(max_depth = 2, n_estimators = bst_n_estimators)
gbrt_best.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

decision_tree_reg = DecisionTreeRegressor(random_state = 0)
cross_val_score(decision_tree_reg, X_train, y_train, cv = 10)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

mlp_reg = MLPRegressor(random_state=0, max_iter=500).fit(X_train, y_train)

mlp_reg.score(X_test, y_test)